In [1]:
# !pip install selenium

In [2]:
# ! pip install webdriver_manager

In [3]:
import time
from threading import Thread
import logging
import os
from datetime import datetime
import re

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait, Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains

print("Working")

Working


In [4]:
# ------------- Global Vars -------------
# CWD = os.path.dirname(os.path.realpath(__file__))
CWD = os.path.dirname(os.path.realpath("./"))
tdt = datetime.now()
td = datetime.now().strftime('%m-%d-%Y')
FIVEM_URL = 'https://servers.fivem.net/servers'
BASE_URL = 'https://servers.fivem.net/servers/detail/{}'
FIVEM_OUTPUT = os.path.join(CWD, 'FiveMScraper.csv')

print("Working")

Working


In [5]:

# ------------- Logging Configuration -------------
log_file_path = os.path.join(CWD, f"fivem_scraper.log")
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
file_handler = logging.FileHandler(filename=log_file_path, mode="w", encoding="UTF-8")
file_handler.setFormatter(
    logging.Formatter("%(asctime)s-%(levelname)s: Func-%(funcName)s : Line-%(lineno)d : %(message)s"))
logger.addHandler(file_handler)
logger.addHandler(logging.StreamHandler())
logger.debug(f"\n{'-' * 21} FiveM Scraper == Started at -> {tdt.strftime('%d-%m-%Y %I:%M:%S %p')} {'-' * 21}")
logger.info(f"Current Working Directory: {CWD}")
logger.info(f"Current Date & Time: {tdt.strftime('%d-%m-%Y %I:%M:%S %p')}")
logger.info(f"Log File Path: {log_file_path}")
logger.info(f"OUTPUT File Path: {FIVEM_OUTPUT}")
# ------------- Starter Code -------------
logging.getLogger('WDM').setLevel(logging.CRITICAL)
os.environ['WDM_LOG'] = '0'
# DRIVER_PATH = ChromeDriverManager(version="114.0.5735.90").install()
# logger.info(f"Chrome Driver File Path: {log_file_path}")



--------------------- FiveM Scraper == Started at -> 23-10-2023 12:24:54 PM ---------------------
Current Working Directory: C:\Users\muham\Desktop
Current Date & Time: 23-10-2023 12:24:54 PM
Log File Path: C:\Users\muham\Desktop\fivem_scraper.log
OUTPUT File Path: C:\Users\muham\Desktop\FiveMScraper.csv


In [6]:

def push_data(data):
    if not os.path.exists(FIVEM_OUTPUT):
        df = pd.DataFrame(
            columns=["Server Name", "FiveM Server Link", "Developer", "Game type", "Map", "OneSync", "Language",
                     "Tags", "DLC level", "sv_purelevel", "Owner", "DiscordLink", "Extracted_Link", "Resources Section"])
        df.loc[df.shape[0]] = data
        df.to_csv(FIVEM_OUTPUT, index=False)  # Save as CSV

    else:
        df = pd.read_csv(FIVEM_OUTPUT)  # Read existing CSV
        df.loc[df.shape[0]] = data
        df.to_csv(FIVEM_OUTPUT, index=False)  # Save as CSV


In [7]:
def get_last_processed_index():
    if os.path.exists(FIVEM_OUTPUT):
        df = pd.read_csv(FIVEM_OUTPUT)
        return df.shape[0]
    return 0

In [8]:
class FiveMScraper:
    def __init__(self):
        """
        To initialize driver and wait objects
        """
        self.main_window = None
        self.index_count = 0
        self.data = ["", "", "", "", "", "", "", "", "", "", "", "", "", ""]
        user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
        
      
        # Create driver and Options
        chrome_options = Options()
        chrome_options.add_argument('--ignore-certificate-errors')
        chrome_options.add_argument(f'user-agent={user_agent}')
        chrome_options.add_argument("--disable-session-crashed-bubble")
        chrome_options.add_argument("--disable-notifications")
        chrome_options.add_argument("--suppress-message-center-popups")
#         chrome_options.add_argument('--disable-logging')
#         chrome_options.add_argument("--disable-translate")
        chrome_options.add_argument("--no-first-run")
        chrome_options.add_argument("--disable-extensions")
        chrome_options.add_argument("--disable-blink-features=AutomationControlled")
        chrome_options.add_argument("disable-infobars")
        #headless
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--disable-gpu')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument("--start-maximized")
        chrome_options.add_argument("--window-size=1920x1080")
        #headless
        chrome_options.add_experimental_option("excludeSwitches", ["enable-automation", "enable-logging"])
        chrome_options.add_experimental_option('useAutomationExtension', False)
        chrome_options.add_experimental_option("excludeSwitches", ["enable-logging", "test-type"])
        
        self.__driver = webdriver.Chrome(service=Service(r"chromedriver.exe"), options=chrome_options)
        self.__wait = WebDriverWait(self.__driver, 60)        
        self.__action = ActionChains(self.__driver)

    def fetch_data(self, server_id):
        url = BASE_URL.format(server_id)
        logger.info(f"{url}")
        self.__driver.switch_to.window("review_details")
        self.__driver.get(url)

        try:
            self.data[1] = url
            snamez = self.__driver.find_element(By.XPATH, '(//span)[24]')
            # //div[@class="info"]//div[@class="title"][1]
            server_namez = snamez.text
            print(server_namez)

            pattern = r"discord\.gg/[a-zA-Z0-9]+(?:\.[a-zA-Z]+)?"
            links = re.findall(pattern, server_namez)
            if links:
                print("Found link(s):")
                print(links)
                self.data[12] = links
            else:
                print("no links")
                self.data[12] = " "
        except:
            pass

        resource_list = []
        resources = self.__wait.until(
            EC.presence_of_all_elements_located((By.XPATH, '(//ul)[3]/li')))
        for resource in resources:
            resource_list.append(resource.text)
            #logger.info(f"Resource: {resource.text}")
        self.data[13] = resource_list
        logger.info(f"Resources: {', '.join(resource_list)}")

        # player_list = []
        # players = self.__wait.until(
        #     EC.presence_of_all_elements_located((By.XPATH, '(//ul)[4]/li')))
        # for player in players:
        #     player_list.append(player.text)
        #     logger.info(f"Player: {player.text}")
        
#         self.data[1] = url

        # time.sleep(5)
        tags = self.__wait.until(
            EC.presence_of_all_elements_located((By.XPATH, '(//ul)[2]/li')))
        # for index in range(len(tags)):
        for tag in tags:
            # key = self.__wait.until(
            #     EC.presence_of_element_located((By.XPATH, f'(//ul)[2]/li[{index + 1}]/span[1]')))
            # value = self.__wait.until(
            #     EC.presence_of_element_located((By.XPATH, f'(//ul)[2]/li[{index + 1}]/span[2]')))
            key = WebDriverWait(tag, 60).until(
                EC.presence_of_element_located((By.XPATH, './span[1]')))

            value = WebDriverWait(tag, 60).until(
                EC.presence_of_element_located((By.XPATH, './span[2]')))

            # ["Server Name", "FiveM Server Link", "Resources Section", "Game type", "Map", "OneSync", "Language",
            #  "Tags", "DLC level", "sv_purelevel", "Owner", "DiscordLink", "Developer"]
            key = key.text
            value = value.text
            logger.info(f"{key}: {value}")
            if key == 'Game type':
                self.data[3] = value
            elif key == 'OneSync':
                self.data[4] = value
            elif key == 'Map':
                self.data[5] = value
            elif key == 'Language':
                self.data[6] = value
            elif key == 'Tags':
                self.data[7] = value
            elif key == 'DLC level':
                self.data[8] = value
            elif key == 'sv_purelevel':
                self.data[9] = value
            elif key == 'Owner':
                self.data[10] = value
            elif key == 'Developer':
                self.data[2] = value
            elif key == 'DiscordLink':
                self.data[11] = value
            # logger.info(f"Tag: {tag.text}")

        print("=========================>")
        print(self.data[6])
#         if "english" in self.data[6].lower():
#             print("The language is English, now pushing data")
#             push_data(self.data)

        push_data(self.data)
        self.data = ["", "", "", "", "", "", "",
                     "", "", "", "", "", "",""]
        self.__driver.switch_to.window(self.__driver.window_handles[0])

        
        
    def start(self):
        print("Waiting to open and load")
        self.__driver.get(FIVEM_URL)
        
        self.main_window = self.__driver.current_window_handle
        self.__driver.execute_script("window.open('about:blank','review_details');")

        server_list = self.__wait.until(
            EC.presence_of_all_elements_located((By.XPATH, '//servers-list[1]//servers-list-item/a/figure/img')))
        titles = self.__wait.until(
            EC.presence_of_all_elements_located((By.XPATH, '//servers-list[1]//servers-list-item/a/div[4]/strong/span')))
        
        print(len(server_list))
        
        server_ids = []

        for index in range(len(server_list)):
            # try:
            self.data = ["", "", "", "", "", "", "", "", "", "", "", "", "", ""]
            image_url = server_list[index].get_attribute('src')
            logger.info(f"{image_url}")
            server_id_ = image_url.split('/')[-2]
            server_name = titles[index].text
            server_ids.append(server_id_)
            logger.info(f"Server Name: {server_name}")
            self.data[0] = server_name
            logger.info(f"Server ID: {server_id_}")
            self.fetch_data(server_id_)
            # except:
            #     print("exception")
            #     pass

print("Updated")

Updated


In [9]:
if __name__ == "__main__":
    bot = FiveMScraper()
    bot.start()

Waiting to open and load


https://servers-live.fivem.net/servers/icon/pk54ay/-1591134233.png
Server Name: Envy Roleplay Serious Roleplay Server
Server ID: pk54ay
https://servers.fivem.net/servers/detail/pk54ay


42


Resources: _EDA_01, 17mov_BuilderJob, 17mov_Deliverer, 17mov_GarbageCollector, 17mov_TreasureHunter, ak4y-advancedFishing, ak4y-advancedHunting, ak4y-caseOpening, ak4y-dailyRewards, ak4y-playTimeShop, ak4y-treasurehunting, ak4y-vipSystem, an_dialogBox, av_laptop, av_music, aw3-ui, baseevents, basic-gamemode, beanmachine, bob74_ipl, boii-pawnshop, brofx_mansion_03, brofx_mansion_19, brofx_villa_21, brofx06, brother_lighthouse, busclothing, busra_animmenu, bz-carpack, bz-carsfxounds, bz-peds, casinohack, cfx_gn_cluckin_rockford, cfx_gn_collection, cfx-gabz-ammunation, cfx-gabz-aztecas, cfx-gabz-bahama, cfx-gabz-barber, cfx-gabz-bobcat, cfx-gabz-bowling, cfx-gabz-burgershot, cfx-gabz-carmeet, cfx-gabz-casino, cfx-gabz-catcafe, cfx-gabz-hayes, cfx-gabz-mapdata, cfx-gabz-paletobank, cfx-gabz-pdprops, cfx-gabz-pizzeria, cfx-gabz-ponsonbys, cfx-gabz-prison, cfx-gabz-suburban, cfx-gabz-tattoo, cfx-gabz-townhall, cfx-gabz-tuners, cfx-gabz-vagos, cfx-gabz-vbmarket, cfx-gabz-yachts, chat-theme-gt

=========================>
English (United States)


...

=========================>
Portuguese (Brazil)


Resources: baixada_arena, baixada_arsenal, baixada_bank, baixada_call, baixada_carros, baixada_char, baixada_dealership, baixada_defuse, baixada_facs, baixada_garages, baixada_hud, baixada_interactions, baixada_inventory, baixada_jobs, baixada_mapas, baixada_mdt, baixada_module, baixada_pvp, baixada_recruitment, baixada_robberies, baixada_roupas, baixada_skinshop, cfx-nikit0-weapons-1, chat, christmas, combo_race, complexo_newbox, core_misc, core_roubos, de_mirage, dom_module, dominacao, du_maps, favelaskillua, furia_painel, furia_sound, lafy, mirtin_craft_v2, mirtin_esc, mirtin_homes, mirtin_survival, MQCU, neons, oxmysql, painel_hospital, pma-voice, scanner, skinweapon, smartphone, smartphone-jobs, tunning, vrp, vrp_admin, vrp_barbershop, vrp_fanho, vrp_legacyfuel, vrp_notifypush, vrp_sounds, vrp_taskbar, vrp_tattoos, vrp_tencode, vrpserver, xsound, zo_attachs
OneSync: enabled
Language: Portuguese (Brazil)
Tags: br,vrp
Desenvolvedor: Killua#0001
Discord: discord.gg/baixadasp
DLC leve

=========================>
Portuguese (Brazil)


Resources: 03cobra, 5MDEVS_furnitureV3Loaded, 5MDEVS_furnitureV4, 5MDevs_luxemanor, 5MDevs_V1usable, 23Escvs, 23tundratrd, 86b64, 88ss30s, 300c, 812gts, 2019zr1SC, A1CT5, ak47_anklemonitor, ak47_cannabiscafev2, ak47_cardealer, ak47_iclothingv2, ak47_lasersight, ak47_smokingv2, ak47_whitewidowv2, AlfaGiulia21, awcustomhell, awjusta6, Badger_Discord_API, banshee87, baseevents, BCall_progress, benihana, Better-AD, bigbennyblackwhite, blips_creator, bmwm4csl, boostlam, brnx_hautenightclub, brutal_banking, brutal_gym, brutal_pets, brzcus, bt-polyzone, btracky, Buty_InfiniteJobs, c63s2023, ccm3, cd_drawtextui, cd_easytime, cd_vehiclecontrol, cfx-gabz-247, cfx-gabz-binco, cfx-gabz-firedept, cfx-gabz-mapdata, cfx-gabz-pdm, charger23d, chargerf8, chat, chevy, cinematiccam, core_crafting, core_dispatch, core_mdw, crazynova, cron, cs_radio, cumminschevyjd, Custom_clothing, datacrack, demod_e92bagged, DemonCharger21SCV3, devcore_smokev2, DiscordAcePerms, Dodge_MoparV2, doitdigital_50_weapons_insan

=========================>
English (United States)


In [5]:
lines = {'Readme', 'How to write text files in Python'}

In [7]:
with open('setLinks.txt', 'w') as f:
    for line in lines:
        f.write(line)
        f.write('\n')